In [6]:
import pandas as pd
import requests
import json
import time
from tqdm import tqdm

def getGameIds():
    headers = {
        "x-api-key" : "0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z"
    }
    resultlist = []
    tournaments = pd.read_excel("../data/target_tournament.xlsx")["id"]
    for tournament in tqdm(tournaments):
        apiResult = requests.get(f"https://prod-relapi.ewp.gg/persisted/gw/getCompletedEvents?hl=en-US&tournamentId={tournament}", headers=headers)
        if apiResult.status_code == 200:
            json = apiResult.json()
            if 'data' in json and len(json['data']) > 0:
                events = json["data"]['schedule']['events']
                for event in events:
                    match = event["match"]
                    games = event["games"]
                    blueteam = match["teams"][0]
                    redteam = match["teams"][1]
                    for idx, game in enumerate(games):
                        if "vods" in game and len(game["vods"]) > 0:
                            game_data = {
                                "startTime(match)" : event["startTime"][:19]+"Z",
                                "blockName" : event["blockName"],
                                "leagueName": event["league"]["name"],
                                "matchId": match["id"],
                                "bestof": match["strategy"]["count"],
                                "blueteam_name" : blueteam["name"],
                                "blueteam_code" : blueteam["code"],
                                "blueteam_win" : blueteam["result"]["gameWins"],
                                "redteam_name" : redteam["name"],
                                "redteam_code" : redteam["code"],
                                "redteam_win" : redteam["result"]["gameWins"],
                                "gameNumberInAMatch": idx,
                                "gameId": game["id"]
                            }
                            resultlist.append(game_data)
        time.sleep(1)
    game_ids_df = pd.DataFrame(resultlist)
    return game_ids_df

In [7]:
game_ids = getGameIds() # 게임아이디가 들어간 리스트
game_ids

100%|██████████| 189/189 [05:25<00:00,  1.72s/it]


,startTime(match),blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,redteam_code,redteam_win,gameNumberInAMatch,gameId
0,2024-05-01T08:00:00Z,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,PSG,1,0,112274687401036075
1,2024-05-01T08:00:00Z,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,PSG,1,1,112274687401036076
2,2024-05-01T08:00:00Z,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,PSG,1,2,112274687401036077
3,2024-05-01T11:00:00Z,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,EST,0,0,112274687401036079
4,2024-05-01T11:00:00Z,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,EST,0,1,112274687401036080
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9288,2023-04-22T10:00:00Z,Knockouts,VCS,109749565998970929,5,CERBERUS ESPORTS,CES,0,SBTC ESPORTS,SE,3,2,109749565998970932
9289,2023-04-23T10:00:00Z,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,GAM,3,0,109749565999036472
9290,2023-04-23T10:00:00Z,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,GAM,3,1,109749565999036473
9291,2023-04-23T10:00:00Z,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,GAM,3,2,109749565999036474


In [9]:
game_ids.to_excel("../data/game_ids.xlsx", index=None)
game_ids

,startTime(match),blockName,leagueName,matchId,bestof,blueteam_name,blueteam_code,blueteam_win,redteam_name,redteam_code,redteam_win,gameNumberInAMatch,gameId
0,2024-05-01T08:00:00Z,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,PSG,1,0,112274687401036075
1,2024-05-01T08:00:00Z,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,PSG,1,1,112274687401036076
2,2024-05-01T08:00:00Z,Play-Ins,MSI,112274687401036074,3,FlyQuest,FLY,2,PSG Talon,PSG,1,2,112274687401036077
3,2024-05-01T11:00:00Z,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,EST,0,0,112274687401036079
4,2024-05-01T11:00:00Z,Play-Ins,MSI,112274687401036078,3,T1,T1,2,Estral Esports,EST,0,1,112274687401036080
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9288,2023-04-22T10:00:00Z,Knockouts,VCS,109749565998970929,5,CERBERUS ESPORTS,CES,0,SBTC ESPORTS,SE,3,2,109749565998970932
9289,2023-04-23T10:00:00Z,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,GAM,3,0,109749565999036472
9290,2023-04-23T10:00:00Z,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,GAM,3,1,109749565999036473
9291,2023-04-23T10:00:00Z,Finals,VCS,109749565999036471,5,SBTC ESPORTS,SE,1,GAM Esports,GAM,3,2,109749565999036474
